In [1]:
require 'openai'
require 'dotenv/load'

def load_env(env_path)
  Dotenv.load(env_path) ? "Environment variables loaded" : "Environment variables not loaded"
end

def openai_client
  @openai_client ||= OpenAI::Client.new(access_token: ENV["OPENAI_API_KEY"], request_timeout: 100000)
end

def sample_files_path
  @sample_files_path ||= "ruby-openai/sample_files"
end

def files_rb
  @files_rb ||= Dir.glob("#{sample_files_path}/*")
end

def files
  @files = files_rb.map { |file| { file: { name: file, content: File.read(file) } } }
end

def message(code)
  {
    role: "user",
    content: "Generate RSpec tests for the following code:
    \n#{code}\n"
  }
end

def chat_paramaters(message, max_tokens)
  {
    model: "gpt-3.5-turbo",
    messages: [message],
    max_tokens: max_tokens,
    temperature: 0.1,
    top_p: 1,
    frequency_penalty: 0,
    presence_penalty: 0
  }
end

def generate_tests_with_chat(files, openai)
  Dir.mkdir("ruby-openai/rspec") unless Dir.exist?("ruby-openai/rspec")

  files.each do |file|
    name = file[:file][:name]
    content = file[:file][:content]
    puts "Generating tests for #{name}..."
      
    message = message(content)
    chat_paramaters = chat_paramaters(message, 1000)
    begin
      response = openai.chat(parameters: chat_paramaters)
      tests = response["choices"][0]["message"]["content"]
      File.open("ruby-openai/rspec/#{name.split("/").last.split(".").first}_spec.rb", "w") { |f| f.write(tests) }
    rescue StandardError => e
      puts "Error: #{e}"
      retry
    end
  end
end

:generate_tests_with_chat

In [2]:
load_env("ruby-openai/.env")
"Done generating tests" if generate_tests_with_chat(files, openai_client)

Generating tests for ruby-openai/sample_files/subtract.rb...
Generating tests for ruby-openai/sample_files/sum.rb...


"Done generating tests"